<b><font size="6" color="red">ch14. 웹데이터 수집</font></b>

# 1절. BeautifulSoup과 parser
`pip install bs4` 아나콘다 설치하면 자동 설치되는 패키지 7500개에 포함

`pip install requests_file`

공식 사이트 : https://www.crummy.com/software/BeautifulSoup <br>
Documentation : https://www.crummy.com/software/BeautifulSoup/bs4/doc

In [1]:
import requests
from requests_file import FileAdapter

In [2]:
s = requests.Session()
s.mount("file://", FileAdapter())
response = s.get('file:///C:/IVS/C_Python/source/Python/ch12_sample.html')
#C:\IVS\C_Python\source\Pythonch12_sample.html
response  #200뜨면 정상, 404뜨면 오류

<Response [200]>

In [3]:
if response.status_code == 200:
    print('success!')
if response.status_code == 404:
    print('Not found')

success!


In [4]:
response.content.decode('utf-8')

'<html>\n    <head>\n        <title>HTML sample</title>\n        <style>\n            span {color:red;}\n        </style>\n    </head>\n    <body>\n        <h1 class="greeting css" id="test">Hello CSS</h1>\n        <h1 class="css">Hi CSS</h1>\n        <div id="subject">선택자</div>\n        <p>CSS 선택자는 다양한 곳에 활용됩니다</p>\n        <div class="contents">선택자를 어떻게 작성하느냐에 따라 \n\t<span>다른 <b>요소가 반환</b></span>됩니다.\n        </div>\n        <div>CSS선택자는 다양한 곳에 <b>활용</b>됩니다</div>\n        <input type="text" name="passwd" id="passwd">\n    </body>\n</html>\n\n\n\n'

In [5]:
# html 파싱
from bs4 import BeautifulSoup # 클래스 이름
soup=BeautifulSoup(response.content,"html.parser") #xml은?
soup

<html>
<head>
<title>HTML sample</title>
<style>
            span {color:red;}
        </style>
</head>
<body>
<h1 class="greeting css" id="test">Hello CSS</h1>
<h1 class="css">Hi CSS</h1>
<div id="subject">선택자</div>
<p>CSS 선택자는 다양한 곳에 활용됩니다</p>
<div class="contents">선택자를 어떻게 작성하느냐에 따라 
	<span>다른 <b>요소가 반환</b></span>됩니다.
        </div>
<div>CSS선택자는 다양한 곳에 <b>활용</b>됩니다</div>
<input id="passwd" name="passwd" type="text"/>
</body>
</html>

In [6]:
el = soup.select_one('h1') #처음 하나만 가져옴
print(el)
print('el.text :', el.text) #모든 태그를 뺀 텍스트만
print('el의 속성들 :', el.attrs)
print('el의 class속성 :', el.attrs['class'])
print('el의 name :', el.name) #태그이름

<h1 class="greeting css" id="test">Hello CSS</h1>
el.text : Hello CSS
el의 속성들 : {'class': ['greeting', 'css'], 'id': 'test'}
el의 class속성 : ['greeting', 'css']
el의 name : h1


In [7]:
# h1태그들 다 : 없는 태그는 빈스트링. 한 엘리먼트만 있어도 리스트로 반환
el = soup.select("h1")
print('el : ', el)
print('el의 text들 :', [e.text for e in el])
print('el의 string :', [e.string for e in el])
print('el의 속성들 :', [e.attrs for e in el])
print('el의 class속성들 :', [e.attrs['class'] for e in el])

el :  [<h1 class="greeting css" id="test">Hello CSS</h1>, <h1 class="css">Hi CSS</h1>]
el의 text들 : ['Hello CSS', 'Hi CSS']
el의 string : ['Hello CSS', 'Hi CSS']
el의 속성들 : [{'class': ['greeting', 'css'], 'id': 'test'}, {'class': ['css']}]
el의 class속성들 : [['greeting', 'css'], ['css']]


In [8]:
# h1태그들 다 : 없는 태그는 빈스트링. 한 엘리먼트만 있어도 리스트로 반환
el = soup.select('div.contents')
print('el : ', el)
print('el의 text들 :', [e.text for e in el])
print('el의 string :', [e.string for e in el])
print('el의 속성들 :', [e.attrs for e in el])
print('el의 class속성들 :', [e.attrs['class'] for e in el])

el :  [<div class="contents">선택자를 어떻게 작성하느냐에 따라 
	<span>다른 <b>요소가 반환</b></span>됩니다.
        </div>]
el의 text들 : ['선택자를 어떻게 작성하느냐에 따라 \n\t다른 요소가 반환됩니다.\n        ']
el의 string : [None]
el의 속성들 : [{'class': ['contents']}]
el의 class속성들 : [['contents']]


In [9]:
el = soup.select('div.contents')
el = soup.select_one('div.contents')
print(el.text)

선택자를 어떻게 작성하느냐에 따라 
	다른 요소가 반환됩니다.
        


# 2절. 정적 웹 데이터 수집
## 2.1 request 모듈 : 네이버 환율정보 가져오기

In [10]:
import requests
from bs4 import BeautifulSoup
from numpy import round
url = "https://finance.naver.com/marketindex"
marketindex = requests.get(url)
soup = BeautifulSoup(marketindex.content, "html.parser")
price = soup.select("div.head_info > span.value")
for idx in range(len(price)):
    print(price[idx].text)

1,374.60
880.48
1,499.14
189.42
156.1800
1.0869
1.2771
104.0700
74.22
1667.78
2369.3
103746.18


In [11]:
marketindex.content  # html 파일

b'\n<script language=javascript src="/template/head_js.naver?referer=info.finance.naver.com&menu=marketindex&submenu=market"></script>\n<script type="text/javascript" src="https://ssl.pstatic.net/imgstock/static.pc/20240530183512/js/info/jindo.min.ns.1.5.3.euckr.js"></script>\n<script type="text/javascript" src="https://ssl.pstatic.net/imgstock/static.pc/20240530183512/js/jindo.1.5.3.element-text-patch.js"></script>\n\n\n\t\n\t\n\t<div id="container" style="padding-bottom:0px;">\n\t\n\n\t\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t<div class="market_include">\n\t<div class="market_data">\n\t\t<div class="market1">\n\t\t\t<div class="title">\n\t\t\t\t<h2 class="h_market1"><span>\xc8\xaf\xc0\xfc \xb0\xed\xbd\xc3 \xc8\xaf\xc0\xb2</span></h2>\n\t\t\t</div>\n\t\t\t<!-- data -->\n\t\t\t<div class="data">\n\t\t\t\t<ul class="data_lst" id="exchangeList">\n\t\t\t\t\n\t\t\t\t    \n\t\t\t\t         \n\t\t\t\t         \n\t\t\t\t         \n\t\t\t\t         \t\t\t\t         \t\t\t     

In [12]:
title = soup.select('h3.h_lst > span.blind')
for idx in range(len(title)):
    print(title[idx].text)

미국 USD
일본 JPY(100엔)
유럽연합 EUR
중국 CNY
달러/일본 엔
유로/달러
영국 파운드/달러
달러인덱스
WTI
휘발유
국제 금
국내 금


In [13]:
unit = soup.select('div.head_info > span > span.blind')
unit = [u.text for u in unit]
unit.insert(7,'')
unit

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [14]:
status = soup.select('div.head_info > span.blind')
for idx in range(len(status)):
    print(status[idx].text, end=', ')

상승, 상승, 상승, 하락, 하락, 상승, 상승, 하락, 하락, 하락, 상승, 상승, 

In [15]:
len(title), len(price), len(unit), len(status)

(12, 12, 12, 12)

In [16]:
for idx in range(len(title)):
    print("{} : {}{} - {}".format(title[idx].text, price[idx].text, 
                                 unit[idx], status[idx].text))

미국 USD : 1,374.60원 - 상승
일본 JPY(100엔) : 880.48원 - 상승
유럽연합 EUR : 1,499.14원 - 상승
중국 CNY : 189.42원 - 하락
달러/일본 엔 : 156.1800엔 - 하락
유로/달러 : 1.0869달러 - 상승
영국 파운드/달러 : 1.2771달러 - 상승
달러인덱스 : 104.0700 - 하락
WTI : 74.22달러 - 하락
휘발유 : 1667.78원 - 하락
국제 금 : 2369.3달러 - 상승
국내 금 : 103746.18원 - 상승


# 2.2 yest24의 베스트 셀러를 출력

In [20]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
url = "http://www.yes24.com/24/category/bestseller"
# bestseller = requests.get(url)
# soup = BeautifulSoup(bestseller.content, "html.parser")
bestseller = urlopen(url)
soup = BeautifulSoup(bestseller, "html.parser")
authors_ = soup.select("div.info_row > span.info_auth.authPub")
for idx in range(len(el)):
    print(el[idx].select_one('a').text, end="\n\n")

KeyError: 0

In [18]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
url = "http://www.yes24.com/24/category/bestseller"
# bestseller = requests.get(url)
# soup = BeautifulSoup(bestseller.content, "html.parser")
bestseller = urlopen(url)
soup = BeautifulSoup(bestseller, "html.parser")
titles=soup.select("div.item_info > div.info_row > a.gd_name")
# authors = soup.select("div.info_row > span.info_auth.authPub > a")
authors_wrap = soup.select("div.info_row > span.info_auth.authPub")
prices = soup.select("div.info_row > strong.txt_num > em.yes_b")
# 화면출력
print("순위,책이름,저자및출판사,가격")
title = []
author = []
price = []
for idx in range(len(titles)):
    authors = authors_wrap[idx].select_one('a')
    print("{},{},{},{}".format(idx+1, titles[idx].text, authors.text,  prices[idx].text))
    # print("{},{},{},{}".format(idx+1, titles[idx].text, authors[idx].text,  prices[idx].text))
    title.append(titles[idx].text)
    author.append(authors.text)
    price.append(prices[idx].text)

순위,책이름,저자및출판사,가격
1,선재 업고 튀어 대본집 세트,이시은,45,000
2,리틀 라이프 1,한야 야나기하라,13,320
3,리틀 라이프 2,한야 야나기하라,13,320
4,하루 한 장 나의 어휘력을 위한 필사 노트,유선경,21,420
5,불변의 법칙,모건 하우절,22,500
6,나를 소모하지 않는 현명한 태도에 관하여,마티아스 뇔케,16,020
7,쿠이 료코 낙서집 데이드림 아워,쿠이 료코,18,000
8,나는 다정한 관찰자가 되기로 했다,이은경,16,020
9,세이노의 가르침,세이노(SayNo),6,480
10,부자의 마지막 가르침,다우치 마나부,17,820
11,신화의 숲,김헌,16,200
12,컬러의 세계,찰스 브라메스코,17,820
13,원피스 ONE PIECE 108,오다 에이이치로,4,950
14,흔한남매 과학 탐험대 10 생태계,흔한남매,13,320
15,마흔에 읽는 쇼펜하우어,강용수,15,300
16,국어 잘하는 아이가 이깁니다,나민애,15,750
17,THE MONEY BOOK 더 머니북,토스,19,800
18,룬의 아이들 - 블러디드 7,전민희,15,300
19,꽃길이 따로 있나, 내 삶이 꽃인 것을,오평선,15,120
20,나는 메트로폴리탄 미술관의 경비원입니다,패트릭 브링리,15,750
21,돈의 심리학 (30만 부 기념 스페셜 에디션) ,모건 하우절,17,820
22,설민석의 한국사 대모험 29,설민석,11,700
23,빨모쌤의 라이브 영어회화,신용하,19,800
24,나의 돈키호테,김호연,16,200


In [21]:
import pandas as pd
df = pd.DataFrame({'순위':range(1,25),
                   '책이름':title,
                   '저자및출판사':author,
                   '가격':price})
df

,순위,책이름,저자및출판사,가격
0,1,선재 업고 튀어 대본집 세트,이시은,"45,000"
1,2,리틀 라이프 1,한야 야나기하라,"13,320"
2,3,리틀 라이프 2,한야 야나기하라,"13,320"
3,4,하루 한 장 나의 어휘력을 위한 필사 노트,유선경,"21,420"
4,5,불변의 법칙,모건 하우절,"22,500"
5,6,나를 소모하지 않는 현명한 태도에 관하여,마티아스 뇔케,"16,020"
6,7,쿠이 료코 낙서집 데이드림 아워,쿠이 료코,"18,000"
7,8,나는 다정한 관찰자가 되기로 했다,이은경,"16,020"
8,9,세이노의 가르침,세이노(SayNo),"6,480"
9,10,부자의 마지막 가르침,다우치 마나부,"17,820"


In [22]:
df.to_csv('data/yes24.csv', encoding='utf-8', index=False)